In [1]:
# Import necessary libraries
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.model_selection import train_test_split

In [40]:
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load the dataset
data = pd.read_csv('data/train.csv')

# Check column names and ensure the target column exists
print("Columns in dataset:", data.columns)
if 'SalePrice' not in data.columns:
    raise ValueError("Target column 'SalePrice' not found in the dataset. Please check the column names.")

# Separate features and target
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

print("Numeric Columns:", numeric_cols)
print("Categorical Columns:", categorical_cols)

# Define preprocessing pipelines
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

# Get feature names
categorical_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)
processed_features = list(numeric_cols) + list(categorical_features)

# Convert transformed data to a DataFrame
X_df = pd.DataFrame(X_transformed, columns=processed_features)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# Define the Bayesian model
# Define the Bayesian model
with pm.Model() as housing_model:
    # Priors for coefficients
    coefficients = pm.Normal('coefficients', mu=0, sigma=10, shape=X_train.shape[1])
    intercept = pm.Normal('Intercept', mu=0, sigma=10)

    # Define the linear model
    mu = pm.math.dot(X_train.values, coefficients) + intercept

    # Likelihood function
    sigma = pm.HalfNormal('sigma', sigma=10)
    price_obs = pm.Normal('Price', mu=mu, sigma=sigma, observed=y_train.values)

    # Sampling using MCMC
    trace = pm.sample(1000, tune=1000, random_seed=42, cores=1)


# Summarize posterior distributions
summary = pm.summary(trace)
print(summary)


Columns in dataset: Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'Gara

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [coefficients, Intercept, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 17 seconds.


                       mean      sd    hdi_3%   hdi_97%  mcse_mean  mcse_sd  \
coefficients[0]      -2.614   9.725   -20.169    15.021      0.196    0.255   
coefficients[1]     -12.831   9.937   -32.259     5.341      0.190    0.204   
coefficients[2]      47.121  10.021    28.681    66.262      0.192    0.138   
coefficients[3]      44.973   9.264    27.286    61.005      0.178    0.128   
coefficients[4]     111.941   9.810    95.242   131.819      0.217    0.155   
...                     ...     ...       ...       ...        ...      ...   
coefficients[242]     3.786   9.988   -15.661    21.146      0.209    0.208   
coefficients[243]   249.677   9.895   231.034   267.476      0.210    0.149   
coefficients[244]    39.136   9.765    19.809    56.551      0.177    0.127   
Intercept           312.882   9.839   294.363   331.341      0.173    0.122   
sigma              8042.239   5.244  8032.893  8052.849      0.100    0.071   

                   ess_bulk  ess_tail  r_hat  
coef

In [54]:
import numpy as np

# Step 1: Convert X_test to NumPy for matrix operations
X_test_np = X_test.values  # Shape: (n_test, n_features)

# Step 2: Extract posterior samples for coefficients and intercept
coefficients_samples = trace.posterior['coefficients'].values  # Shape: (chains, draws, n_features)
intercept_samples = trace.posterior['Intercept'].values  # Shape: (chains, draws)

# Flatten the samples (combine chains and draws)
coefficients_samples = coefficients_samples.reshape(-1, coefficients_samples.shape[-1])  # Shape: (n_samples, n_features)
intercept_samples = intercept_samples.flatten()  # Shape: (n_samples,)

# Debugging Shapes
print("Shape of coefficients_samples:", coefficients_samples.shape)  # (n_samples, n_features)
print("Shape of intercept_samples:", intercept_samples.shape)  # (n_samples,)
print("Shape of X_test_np:", X_test_np.shape)  # (n_test, n_features)

# Step 3: Compute predictions
# Dot product of posterior samples with X_test, plus intercept
predictions = np.dot(coefficients_samples, X_test_np.T) + intercept_samples[:, None]  # Shape: (n_samples, n_test)

# Step 4: Calculate mean and credible intervals for predictions
predicted_mean = predictions.mean(axis=0)  # Mean prediction for each test point
credible_intervals = np.percentile(predictions, [2.5, 97.5], axis=0)  # 95% credible intervals

# Step 5: Display predictions
actual_values = list(y_test)
for i, (mean, ci) in enumerate(zip(predicted_mean, credible_intervals.T)):
    print(f"Test Row {i}: Predicted Mean = {mean:.2f}, 95% CI = ({ci[0]:.2f}, {ci[1]:.2f})")

    print('\tactual_value --> ', actual_values[i])

Shape of coefficients_samples: (2000, 245)
Shape of intercept_samples: (2000,)
Shape of X_test_np: (292, 245)
Test Row 0: Predicted Mean = 8195.11, 95% CI = (8022.67, 8370.02)
	actual_value -->  154500
Test Row 1: Predicted Mean = 9608.64, 95% CI = (9437.44, 9779.37)
	actual_value -->  325000
Test Row 2: Predicted Mean = 6864.16, 95% CI = (6704.17, 7024.52)
	actual_value -->  115000
Test Row 3: Predicted Mean = 7274.16, 95% CI = (7120.77, 7421.94)
	actual_value -->  159000
Test Row 4: Predicted Mean = 8800.19, 95% CI = (8639.92, 8961.87)
	actual_value -->  315500
Test Row 5: Predicted Mean = 6224.28, 95% CI = (6014.54, 6430.42)
	actual_value -->  75500
Test Row 6: Predicted Mean = 8292.80, 95% CI = (8112.62, 8469.87)
	actual_value -->  311500
Test Row 7: Predicted Mean = 8385.57, 95% CI = (8228.44, 8538.68)
	actual_value -->  146000
Test Row 8: Predicted Mean = 6220.30, 95% CI = (6007.30, 6428.04)
	actual_value -->  84500
Test Row 9: Predicted Mean = 7807.54, 95% CI = (7636.76, 7977.14

In [ ]:
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load the dataset
data = pd.read_csv('data/train.csv')

# Check column names and ensure the target column exists
print("Columns in dataset:", data.columns)
if 'SalePrice' not in data.columns:
    raise ValueError("Target column 'SalePrice' not found in the dataset. Please check the column names.")

# Separate features and target
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

print("Numeric Columns:", numeric_cols)
print("Categorical Columns:", categorical_cols)

# Define preprocessing pipelines
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

# Get feature names
categorical_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)
processed_features = list(numeric_cols) + list(categorical_features)

# Convert transformed data to a DataFrame
X_df = pd.DataFrame(X_transformed, columns=processed_features)

# Standardize the target variable
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()  # Flatten for PyMC3

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_scaled, test_size=0.2, random_state=42)


# determine if you want to use a normal distribution for the prices or not
# -------- the normal distribution gives better predictions than the gamma one???
use_normal_distribution_for_prices = False
if use_normal_distribution_for_prices:
    # Define the Bayesian model
    with pm.Model() as housing_model:
        # Priors for coefficients and intercept
        coefficients = pm.Normal('coefficients', mu=0, sigma=10, shape=X_train.shape[1])
        intercept = pm.Normal('Intercept', mu=0, sigma=1000)

        # Define the linear model
        mu = pm.math.dot(X_train.values, coefficients) + intercept

        # Likelihood function
        sigma = pm.HalfNormal('sigma', sigma=10)
        price_obs = pm.Normal('Price', mu=mu, sigma=sigma, observed=y_train)

        # Sampling using MCMC
        trace = pm.sample(1000, tune=1000, random_seed=42, cores=1)

else:
    with pm.Model() as housing_model:
        # Priors for coefficients and intercept
        coefficients = pm.Normal('coefficients', mu=0, sigma=5, shape=X_train.shape[1])  # Constrained prior
        intercept = pm.Normal('Intercept', mu=1, sigma=5)  # Positive mean for better initialization

        # Rescale y_train
        y_train_scaled = (y_train - y_train.min()) + 1e-3  # Ensure values are positive and offset by a small value

        # Define the linear model with a positive constraint
        mu = pm.math.exp(pm.math.dot(X_train.values, coefficients) + intercept)

        # Priors for the shape parameter
        shape = pm.Gamma('shape', alpha=5, beta=1)  # Stable prior for shape

        # Likelihood function (Gamma distribution)
        price_obs = pm.Gamma('Price', alpha=shape, beta=shape / mu, observed=y_train_scaled)

        # Sampling using MCMC with custom initial values
        trace = pm.sample(1000, tune=1000, random_seed=42, cores=1, initvals={
            'coefficients': np.random.normal(0, 0.1, size=X_train.shape[1]),
            'Intercept': 0.1,
            'shape': 2.0
        })







# Summarize posterior distributions
summary = pm.summary(trace)
print(summary)

# Step 1: Transform X_test to NumPy for matrix operations
X_test_np = X_test.values  # Shape: (n_test, n_features)

# Step 2: Extract posterior samples for coefficients and intercept
coefficients_samples = trace.posterior['coefficients'].values.reshape(-1, X_test_np.shape[1])  # Flatten samples
intercept_samples = trace.posterior['Intercept'].values.flatten()  # Flatten intercept samples

# Step 3: Compute predictions for X_test
predictions = np.dot(coefficients_samples, X_test_np.T) + intercept_samples[:, None]  # Shape: (samples, test_rows)

# Step 4: Transform predictions back to the original scale
predicted_mean = predictions.mean(axis=0)  # Mean prediction for each test point
credible_intervals = np.percentile(predictions, [2.5, 97.5], axis=0)  # 95% credible intervals

# Transform back to original scale
predicted_mean_original_scale = target_scaler.inverse_transform(predicted_mean.reshape(-1, 1)).flatten()
credible_intervals_original_scale = target_scaler.inverse_transform(credible_intervals.T).T

# Step 5: Display predictions
for i, (mean, ci) in enumerate(zip(predicted_mean_original_scale, credible_intervals_original_scale.T)):
    print(f"Test Row {i}: Predicted Mean = {mean:.2f}, 95% CI = ({ci[0]:.2f}, {ci[1]:.2f})")
    print(f"Actual Value = {target_scaler.inverse_transform(y_test[i].reshape(-1, 1)).flatten()[0]:.2f}")


Columns in dataset: Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'Gara

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [coefficients, Intercept, shape]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 599 seconds.


                     mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
coefficients[0]    -0.001  0.006  -0.013    0.010      0.000    0.000   
coefficients[1]    -0.054  0.037  -0.122    0.013      0.003    0.002   
coefficients[2]     0.006  0.009  -0.011    0.022      0.001    0.000   
coefficients[3]     0.025  0.011   0.005    0.048      0.001    0.001   
coefficients[4]     0.079  0.013   0.055    0.103      0.001    0.000   
...                   ...    ...     ...      ...        ...      ...   
coefficients[242]   0.061  0.056  -0.044    0.160      0.004    0.003   
coefficients[243]   0.082  0.029   0.024    0.136      0.002    0.002   
coefficients[244]  -0.025  0.226  -0.458    0.370      0.013    0.010   
Intercept          -3.548  0.916  -5.256   -1.872      0.107    0.076   
shape              28.306  1.250  25.803   30.485      0.065    0.046   

                   ess_bulk  ess_tail  r_hat  
coefficients[0]       494.0     750.0   1.00  
coefficients[1]       119.0  

In [85]:
# Initialize a list to store residuals
rss = []

for i, (mean, ci) in enumerate(zip(predicted_mean_original_scale, credible_intervals_original_scale.T)):
    # Get the actual value by transforming the scaled target back to the original scale
    actual_value = target_scaler.inverse_transform(np.array([[y_test[i]]])).flatten()[0]

    # Print predictions and actual values for debugging
    print(f"Test Row {i}: Predicted Mean = {mean:.2f}, 95% CI = ({ci[0]:.2f}, {ci[1]:.2f})")
    print(f"Actual Value = {actual_value:.2f}")

    # Append the squared residual to the list
    rss.append(np.square(actual_value - mean))

# Calculate total RSS
total_rss = np.sum(rss)
print(f"\nTotal RSS: {total_rss:.2f}")


Test Row 0: Predicted Mean = 213993.50, 95% CI = (204883.72, 223025.02)
Actual Value = 154500.00
Test Row 1: Predicted Mean = 290189.57, 95% CI = (280883.00, 300073.74)
Actual Value = 325000.00
Test Row 2: Predicted Mean = 165522.50, 95% CI = (152688.38, 178674.35)
Actual Value = 115000.00
Test Row 3: Predicted Mean = 221528.84, 95% CI = (209126.37, 234609.61)
Actual Value = 159000.00
Test Row 4: Predicted Mean = 279688.77, 95% CI = (267270.19, 293075.19)
Actual Value = 315500.00
Test Row 5: Predicted Mean = 141243.95, 95% CI = (124124.78, 157532.63)
Actual Value = 75500.00
Test Row 6: Predicted Mean = 263959.11, 95% CI = (251985.79, 274749.44)
Actual Value = 311500.00
Test Row 7: Predicted Mean = 206002.54, 95% CI = (189291.50, 222996.87)
Actual Value = 146000.00
Test Row 8: Predicted Mean = 135222.29, 95% CI = (118339.36, 151769.97)
Actual Value = 84500.00
Test Row 9: Predicted Mean = 205761.51, 95% CI = (195893.48, 216136.13)
Actual Value = 135500.00
Test Row 10: Predicted Mean = 20

In [86]:
mse = total_rss / len(y_test)
print(f"Mean Squared Error (MSE): {mse:.2f}")
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Mean Squared Error (MSE): 4506510743.03
Root Mean Squared Error (RMSE): 67130.55


In [87]:
actual_values = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
mean_actual = np.mean(actual_values)
ss_total = np.sum(np.square(actual_values - mean_actual))  # Total Sum of Squares
r2 = 1 - (total_rss / ss_total)
print(f"R-squared (R²): {r2:.4f}")


R-squared (R²): 0.4125


## Why use normal distribution for the priors:
Rationale: Normal priors are used here because they encode the belief that the parameter values are most likely to cluster around a central value (e.g., 0) but with some spread (controlled by the standard deviation, sigma).
- #### Why It's Relevant:
        - Most effects in real-world data tend to cluster around a central value (e.g., the effect of neighborhood or room size). The Normal distribution is flexible, allowing you to encode both uncertainty (using large sigma) and prior knowledge (using a specific mean, mu).